In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU
import tensorflow.keras.utils as ku
from tensorflow.keras.callbacks import EarlyStopping

tf.config.run_functions_eagerly(True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

2024-09-01 12:57:43.732162: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 12:57:44.259077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 12:57:44.454796: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 12:57:44.505757: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 12:57:44.865251: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1


I0000 00:00:1725175668.486117   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725175669.053570   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725175669.053834   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725175669.060725   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725175669.060911   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [2]:
rawdataset = pd.read_csv("./dataset/full_dataset.csv")

In [3]:
dataset = rawdataset[:5000]
dataset = dataset.drop(["link", "source"], axis=1)

In [4]:
print("Data shape:", dataset.shape)
dataset.head()

Data shape: (5000, 5)


,Unnamed: 0,title,ingredients,directions,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [5]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation.replace(",", "")).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

NER = [clean_text(x) for x in dataset.NER]
ingredients = [clean_text(x) for x in dataset.ingredients]
directions = [clean_text(x) for x in dataset.directions]

In [6]:
finaldataset = []
for i in range(len(NER)):
    finaldataset.append(NER[i]+" | "+directions[i])
max_length = 0
for i in finaldataset:
    max_length = max(max_length, len(i))
print("max_length:", max_length)
print()
finaldataset = NER
print("Recipe at index 0:\n", finaldataset[0])

max_length: 1120

Recipe at index 0:
 brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits


In [7]:
tokenizer = Tokenizer(filters='!"#$%&()*+-./:;<=>?@[\\]^_`{}~\t\n')

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1    
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(finaldataset)

inp_sequences[:10], total_words
len(inp_sequences)
#tokenizer.get_config()

47686

In [8]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences[:100212])
predictors,label,max_sequence_len

(array([[ 0,  0,  0, ...,  0,  0, 21],
        [ 0,  0,  0, ...,  0, 21,  1],
        [ 0,  0,  0, ..., 21,  1,  8],
        ...,
        [ 0,  0,  0, ..., 13,  8, 14],
        [ 0,  0,  0, ...,  8, 14, 19],
        [ 0,  0,  0, ..., 14, 19, 77]], dtype=int32),
 array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 69)

In [9]:
def create_model(total_words):
    model = Sequential()
    
    # Setting early_stopping feature to stop early on getting stagnant
    early_stopping = EarlyStopping(
    min_delta=0.01, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
    )
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 100))
   
    # input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
    # output_dim: Integer. Dimension of the dense embedding.
    # input_length: Length of input sequences, when it is constant. 

    # Add Hidden Layer 1 - RNN Layer
    model.add(LSTM(200))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(total_words)
#model.summary()

I0000 00:00:1725175692.455678   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725175692.455793   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725175692.455808   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725175692.456020   68270 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-01 12:58:12.456034: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2112] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.

In [10]:
model.fit(predictors, label, epochs=10)

/home/rahul/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10


2024-09-01 12:58:15.276047: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


1491/1491 ━━━━━━━━━━━━━━━━━━━━ 63s 40ms/step - loss: 5.5016
Epoch 2/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 58s 39ms/step - loss: 4.1626
Epoch 3/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 58s 39ms/step - loss: 3.7262
Epoch 4/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 57s 38ms/step - loss: 3.4591
Epoch 5/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 58s 39ms/step - loss: 3.2596
Epoch 6/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 59s 39ms/step - loss: 3.0728
Epoch 7/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 60s 40ms/step - loss: 2.9082
Epoch 8/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 58s 39ms/step - loss: 2.7585
Epoch 9/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 63s 42ms/step - loss: 2.6266
Epoch 10/10
1491/1491 ━━━━━━━━━━━━━━━━━━━━ 60s 40ms/step - loss: 2.4980


In [13]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [20]:
generate_text("dark sweet pitted cherries", 10, model, max_sequence_len)

'Dark Sweet Pitted Cherries Pineapple Juice, Lemon Juice, Sugar, Lemon Juice, Lemon Juice, Lemon'

In [19]:
NER[10]

'dark sweet pitted cherries, ginger ale, flavor gelatin, boiling water, almond extract, marshmallows'